In [1]:
import pandas as pd
from utils import read
from search import helpers

from sklearn.feature_extraction.text import TfidfVectorizer

%reload_ext autoreload
%autoreload 2

In [2]:
courses_text = read.read_json_processed('courses-text')
slug_keywords = pd.DataFrame(courses_text).T[['keywords']]
# the index is the course slug
slug_keywords.index.name = 'slug'
slug_keywords.reset_index(inplace=True)

courses = read.read_json_processed('courses')
slug_code_name = pd.DataFrame(courses)[['slug', 'code', 'name']]

df = pd.merge(slug_keywords, slug_code_name)

df['search_string'] = df.keywords + ' ' + df.name + ' ' + df.slug.str.replace('-', '')
df['search_string'] = df.search_string.str.strip()

In [3]:
tfidf = TfidfVectorizer(analyzer='word', preprocessor=helpers.clean_text)
features = tfidf.fit_transform(df.search_string)

In [4]:
helpers.save_df(df, 'courses_df')
helpers.save_sparsemat(features, 'features')
helpers.save_pickle(tfidf, 'tfidf')